In [1]:
import musicbox.theory as th
import musicbox.box as box
import random
import mido
from functools import partial

In [2]:
roman = ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII']
tonleiter = list(th.Tone.all())
progressions = [['I', 'V', 'IV', 'I'],
              ['I', 'IV', 'V', 'IV'],
              ['I', 'I', 'IV', 'V'],
              ['I', 'IV', 'I', 'V'],
              ['I', 'IV', 'V', 'V'],
              ['I', 'VI', 'II', 'V'],
              ['I', 'VI', 'IV', 'V'],
              ['I', 'IV', 'V', 'I'],
              ['I', 'V', 'I'],
              ['VI', 'II', 'V', 'I']]

def setting():
    root = random.choice(list(th.Tone.all()))
    scale_types = ['major', 'minor']
    scale_type = random.choice(scale_types)
    scale = th.Scale(root, scale_type)
    rhythm = [random.choice([True, False]) for _ in range(8)]
    progression = random.choice(progressions)
    chords = [scale.degree(p) for p in progression]
    bpm = random.randint(120, 160)
    return {'root': root, 'scale_type': scale_type, 'scale': scale, 'rhythm': rhythm, 'progression': progression,
           'chords': chords, 'bpm': bpm}

def note_in_tones(note, tones):
    return any(map(lambda t: t in note, tones))

def notes_from_tones(tones, notes):
    return list(filter(partial(note_in_tones, tones=tones), notes))

def weight_notes(note, notes):
    diff = [note.distance(n) for n in notes]
    m = min(diff)
    index = diff.index(m)   
    return [0 if index - i == 0 else 1/abs(index - i)**2 for i,_ in enumerate(notes)]

def random_from_notes(tones, notes, previous_note = None):
    notes = notes_from_tones(tones, notes)
    if previous_note is None:
        return random.choice(notes)
    else:
        return random.choices(notes, weights = weight_notes(previous_note, notes))[0]

def random_from_chord(chord, notes, previous_note = None):
    return random_from_notes(chord.tones, notes, previous_note)



In [3]:
asdf = setting()
print(asdf['root'])
print(asdf['scale_type'])
print(repr(asdf['scale']))
print(asdf['rhythm'])
print(asdf['progression'])
print(asdf['chords'])
print(asdf['bpm'])


A
major
Scale("A", "major")
[True, False, False, True, False, True, False, True]
['I', 'IV', 'V', 'I']
[Chord("A", "maj"), Chord("D", "maj"), Chord("E", "maj"), Chord("A", "maj")]
144


In [4]:
s = asdf['scale']
notes = []
previous_note = None
for ch in asdf['chords']:
    for _ in asdf['rhythm']:
        previous_note = random_from_chord(ch, box.GI30.notes, previous_note)
        notes.append(previous_note)

In [5]:
mid = mido.MidiFile()
track = mido.MidiTrack()
mid.tracks.append(track)
names = mido.get_output_names()

In [6]:
print(names)
for name in names:
    if 'FLUID' in name:
        port = mido.open_output(name)

['FLUID Synth (301):Synth input port (301:0) 128:0', 'Midi Through:Midi Through Port-0 14:0']


In [7]:
track.append(mido.MetaMessage('set_tempo', tempo=mido.bpm2tempo(asdf['bpm'])))
track.append(mido.Message('program_change', program=10, time=0))

In [8]:
for i, note in enumerate(notes):
    track.append(mido.Message('note_on', note=note.midi, velocity=127, time=0))
    track.append(mido.Message('note_off', note=note.midi, velocity=127, time=240))

In [9]:
for msg in mid.play():
    port.send(msg)

In [10]:
for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        print(msg)

Track 0: 
<meta message set_tempo tempo=416667 time=0>
program_change channel=0 program=10 time=0
note_on channel=0 note=76 velocity=127 time=0
note_off channel=0 note=76 velocity=127 time=240
note_on channel=0 note=52 velocity=127 time=0
note_off channel=0 note=52 velocity=127 time=240
note_on channel=0 note=57 velocity=127 time=0
note_off channel=0 note=57 velocity=127 time=240
note_on channel=0 note=52 velocity=127 time=0
note_off channel=0 note=52 velocity=127 time=240
note_on channel=0 note=45 velocity=127 time=0
note_off channel=0 note=45 velocity=127 time=240
note_on channel=0 note=76 velocity=127 time=0
note_off channel=0 note=76 velocity=127 time=240
note_on channel=0 note=61 velocity=127 time=0
note_off channel=0 note=61 velocity=127 time=240
note_on channel=0 note=57 velocity=127 time=0
note_off channel=0 note=57 velocity=127 time=240
note_on channel=0 note=62 velocity=127 time=0
note_off channel=0 note=62 velocity=127 time=240
note_on channel=0 note=57 velocity=127 time=0
n

In [11]:
#port.close()

In [12]:
#mid.save('test.mid')

In [13]:
winter = mido.MidiFile('winter.mid')

In [14]:
for i, track in enumerate(winter.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        print(msg)

Track 0: 
<meta message set_tempo tempo=375000 time=0>
<meta message end_of_track time=0>
Track 1: https://musicboxmaniacs.com
<meta message track_name name='https://musicboxmaniacs.com' time=0>
program_change channel=0 program=10 time=0
note_on channel=0 note=62 velocity=127 time=0
note_off channel=0 note=62 velocity=127 time=240
note_on channel=0 note=65 velocity=127 time=480
note_off channel=0 note=65 velocity=127 time=240
note_on channel=0 note=69 velocity=127 time=480
note_off channel=0 note=69 velocity=127 time=240
note_on channel=0 note=74 velocity=127 time=480
note_off channel=0 note=74 velocity=127 time=240
note_on channel=0 note=77 velocity=127 time=480
note_off channel=0 note=77 velocity=127 time=240
note_on channel=0 note=81 velocity=127 time=480
note_off channel=0 note=81 velocity=127 time=240
note_on channel=0 note=67 velocity=127 time=0
note_off channel=0 note=67 velocity=127 time=240
note_on channel=0 note=70 velocity=127 time=0
note_off channel=0 note=70 velocity=127 t

In [15]:
#for msg in winter.play():
#    port.send(msg)

In [16]:
port.close()